### 1. JSON output of "dctap"

In [9]:
import json
json_string = Path('example01.json').read_text(encoding="UTF-8")
ap = json.loads(json_string)

### 2. ShExC Template in Jinja2

In [10]:
"""
ShEx template in Jinja2.
"""
shex_jinja = open('example01.jinja').read()
for line in shex_jinja.splitlines():
   print(line)

{%- for prefix, uri in namespaces.items() %}
PREFIX {{prefix}}: <{{uri}}>
{%- endfor %}
BASE <http://purl.org/yama/example/mybook/v021/#>

{%- for shape in shapes %}
{{shape.shapeID}} {
  {%- for statement in shape. statement_templates %}
  {{statement.property}} {{statement.propertyID}} 
  {%- if statement.valueNodeType %} {{statement.valueNodeType|upper}}{% endif -%}
  {%- if statement.valueDataType %} {{statement.valueDataType}}{% endif -%}
  {%- if statement.min or statement.max %} { {{-statement.min-}}
        {%- if statement.max -%}
         ,{{statement.max}} 
        {%- endif -%} }
  {%- endif -%}
  {%- if statement.MaxInclusive %} MaxInclusive {{statement.MaxInclusive}} {%- endif -%}
  {%- if statement.MinInclusive %} MinInclusive {{statement.MinInclusive}} {%- endif -%}
  {%- if not loop.last -%} ;{%- endif -%}
  {%- endfor %}
}
{%- endfor %}


### 3. Then, in order to get the following ShExC output:

```
PREFIX ex: <http://ex.example/#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

school:Enrollee {
  foaf:age xsd:integer MinInclusive 13 MaxInclusive 20 ;
  ex:hasGuardian IRI {1,2}
}
```

### 4. Render the ShExC Template with Jinja2

In [11]:
from jinja2 import Template
template = Template(shex_jinja)
shexc_output = template.render(ap)
print(shexc_output)


PREFIX ex: <http://ex.example/#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
BASE <http://purl.org/yama/example/mybook/v021/#>
school:Enrollee {
   ex:hasGuardian IRI {1,2};
   foaf:age xsd:integer MaxInclusive 20 MinInclusive 13
}


### 5. Load the PyShExC library

In [12]:
import pyshexc
from pyshexc.parser_impl.generate_shexj import parse

### 6. Convert ShExC to ShExJ

In [13]:
import json
json_ap=parse(shexc_output)._as_json
parsed = json.loads(json_ap)
print(json.dumps(parsed, indent=4, sort_keys=False))

{
    "type": "Schema",
    "shapes": [
        {
            "type": "Shape",
            "id": "http://school.example/#Enrollee",
            "expression": {
                "type": "EachOf",
                "expressions": [
                    {
                        "type": "TripleConstraint",
                        "predicate": "http://ex.example/#hasGuardian",
                        "valueExpr": {
                            "type": "NodeConstraint",
                            "nodeKind": "iri"
                        },
                        "min": 1,
                        "max": 2
                    },
                    {
                        "type": "TripleConstraint",
                        "predicate": "http://xmlns.com/foaf/0.1/age",
                        "valueExpr": {
                            "type": "NodeConstraint",
                            "datatype": "http://www.w3.org/2001/XMLSchema#integer",
                            "maxinclusive": 20,
   